### Importamos librerías

In [109]:
import os
import tweepy as tw
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
#import statsmodels.api as sm
from datetime import datetime
import seaborn as sns
from tqdm import tqdm
import openpyxl

### Credenciales para el uso de la API


In [110]:
consumer_key= 'mEfV8N05qpt3kVQFP2wD5swU5'
consumer_secret= 'vs7zg1AvjK9GUCcYwbQxjypT0h65XdDHurMoopQXzuqnwoRTNY'
access_token= '1267192889416847362-3bkTrmyabVIbjlkpfZRIfsGD0LqwRv'
access_token_secret= 'Sl7gnnajauTISKig3FWiluwWibJXsgYTstuF0JUJta7Qu'

In [111]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [112]:
# Creamos una función que nos devuelve la información de los tweets que deseamos

def api_tweets(key_words, language, date_start, n_tweets):

  result = tw.Cursor(api.search,
                    q=key_words,
                    lang=language,
                    since=date_start,
                    tweet_mode='extended').items(n_tweets)

  return result

In [115]:
search_words = ("telefonica")
lang = "es"
date_since = "2020-06-01"
n_tweets = 8000
tweets = api_tweets(search_words, lang, date_since, n_tweets)

### Creamos el dataframe

In [117]:
# Creamos un df vacío con las columnas que vamos a completar

df = pd.DataFrame(columns=['text', 'created_at', 'created_at_time', 'created_at_hour', 'retweeted', 'retweet_count', 'favorite_count', 
                           'user_ verified', 'user_id', 'user_name', 'user_location', 'user_notificacion', 'user_followers', 'user_friends', 
                           'user_withheld_in_countries', 'mentions_in_tweet', 'is_reply', 'source_device', 'finished_tweet', 
                           'status_count', 'hashtags_text','hastags_indices', 'hastags_in_tweet'])

i = 0

for tweet in tweets:

    # Convertimos el json de cada uno de los tweets en un diccionario
    tweet_d = dict(tweet._json)

    # Creo un estado para coger el texto completo
    status = api.get_status(tweet_d['id'], tweet_mode="extended")
    try:
      text = status.retweeted_status.full_text
      rt = 'Si'
    except AttributeError:  # Not a Retweet
      text = status.full_text
      rt = 'No'
    
    # Creamos una variable que nos dice el número de menciones en el tweet
    try:
        mentions_in_tweet = len(tweet_d['entities']['user_mentions'])
    except:
        mentions_in_tweet = 0

    # Creamos una variable que nos dice si el tweet es una respuesta
    if tweet_d['in_reply_to_screen_name'] == None:
      reply = 'No'
    else:
      reply = tweet_d['in_reply_to_screen_name']

    # Cogemos la fecha del tweet y dividimos en día y hora
    fecha_completa = str(tweet_d['created_at'])
    fecha = fecha_completa.split(sep = ' ')
    texts_date = ' '.join(elemento for elemento in fecha[0:3])
    texts_time = fecha[3] 

    # Creamos una columna que corresponde con las horas, para representarla
    hora = fecha[3].split(sep = ':')
    texts_hour = int(hora[0])

    # Exportamos los hastag del tweet y las posiciones
    aux = list([i["text"] for i in tweet.entities['hashtags']]) 
    aux_pos = [i["indices"] for i in tweet.entities['hashtags']]
    
    # Creamos una fila con la información que queremos almacenar de cara al análisis
    new_row = {
        
    'text': text,

    'created_at': texts_date,
    'created_at_time': texts_time,
    'created_at_hour': texts_hour,

    'retweeted': rt,

    'retweet_count': tweet_d['retweet_count'],
    'favorite_count': tweet_d['favorite_count'],

    'user_verified': tweet_d['user']['verified'], # ---------------------- ALGO PASA CON ESTO, DEVUELVE TODO NaN
    'user_id': tweet_d['user']['id'],
    'user_name': tweet_d['user']['screen_name'],
    'user_location': tweet_d['user']['location'],
    'user_notificacion': tweet.user.notifications,
    'user_followers': tweet_d['user']['followers_count'],
    'user_friends': tweet_d['user']['friends_count'],
    'user_withheld_in_countries':  tweet.user.withheld_in_countries,

    'mentions_in_tweet': mentions_in_tweet,   
    'is_reply': reply,

    'source_device':tweet.source,
    'finished_tweet': tweet.truncated, # Si está incompleto el tweet (hilo)

    'status_count':tweet.user.statuses_count, # ¿QUÉ ES ESTO? interacciones del usuario tanto tweets como retweets 

    'hashtags_text': aux,
    'hastags_indices': aux_pos,
    'hastags_in_tweet': len(aux)

    }
    
    # Añadimos la info al dataframe en una nueva fila
    df.loc[i] = new_row

    i = i + 1
 

Guardado de seguridad:

In [118]:
df.to_excel('extraccion_0.xlsx')

In [119]:
df.head()

,text,created_at,created_at_time,created_at_hour,retweeted,retweet_count,favorite_count,user_ verified,user_id,user_name,...,user_friends,user_withheld_in_countries,mentions_in_tweet,is_reply,source_device,finished_tweet,status_count,hashtags_text,hastags_indices,hastags_in_tweet
0,"🗣️ ¡𝐓𝐨𝐦𝐚 𝐍𝐨𝐭𝐚!✍️ 🇻🇪 \nMin. de Ecosocialismo, @...",Tue Aug 03,01:46:40,1,Si,139,0,NaN,1386447251380359176,NoheGuzman4,...,534,[],2,No,Twitter for Android,False,3327,[],[],0
1,"🗣️ ¡𝐓𝐨𝐦𝐚 𝐍𝐨𝐭𝐚!✍️ 🇻🇪 \nMin. de Ecosocialismo, @...",Tue Aug 03,01:46:08,1,Si,139,0,NaN,152870048,katyanicarolina,...,337,[],2,No,Twitter for Android,False,55246,[],[],0
2,Los #TrasladosEduFiscal del régimen #SierraAma...,Tue Aug 03,01:45:53,1,Si,12,0,NaN,931643047930138624,MariaEditaCuev2,...,24,[],1,No,Twitter for Android,False,2007,"[TrasladosEduFiscal, SierraAmazonía]","[[22, 41], [54, 69]]",2
3,Recorrimos las obras del nuevo Contact Center ...,Tue Aug 03,01:45:48,1,Si,65,0,NaN,1365607806,AlmaFuerte40,...,1908,[],1,No,Twitter for Android,False,336030,[],[],0
4,Si sufriste o sufris violencia d género La Lín...,Tue Aug 03,01:45:18,1,No,4,2,NaN,3153998255,silloia,...,3561,[],0,silloia,Twitter for Android,False,9998,[],[],0


Limpiar tildes:

In [120]:
def eliminar_tildes(texto):
    texto_limpio = re.sub('(á)|(Á)|(à)|(À)','a', texto)
    texto_limpio = re.sub('(é)|(É)|(è)|(È)','e', texto_limpio)
    texto_limpio = re.sub('(í)|(Í)|(ì)|(Ì)','i', texto_limpio)
    texto_limpio = re.sub('(ó)|(Ó)|(ò)|(Ò)','o', texto_limpio)
    texto_limpio = re.sub('(ú)|(Ú)|(ù)|(Ù)','u', texto_limpio)
    return texto_limpio

In [121]:
df['text']=df['text'].apply(lambda x: eliminar_tildes(x))

Función para ver si es hay una URL en el tweet


In [122]:
#Regular expresion para capturar urls
url_re = "https://[^\s]+"

In [123]:
df['URLs'] = df['text'].apply(lambda x: re.findall(url_re, x))
df['text'] = df['text'].apply(lambda x: re.sub(url_re, 'url', x))

In [124]:
df

,text,created_at,created_at_time,created_at_hour,retweeted,retweet_count,favorite_count,user_ verified,user_id,user_name,...,user_withheld_in_countries,mentions_in_tweet,is_reply,source_device,finished_tweet,status_count,hashtags_text,hastags_indices,hastags_in_tweet,URLs
0,"🗣️ ¡𝐓𝐨𝐦𝐚 𝐍𝐨𝐭𝐚!✍️ 🇻🇪 \nMin. de Ecosocialismo, @...",Tue Aug 03,01:46:40,1,Si,139,0,NaN,1386447251380359176,NoheGuzman4,...,[],2,No,Twitter for Android,False,3327,[],[],0,[https://t.co/CZpTz7xDqU]
1,"🗣️ ¡𝐓𝐨𝐦𝐚 𝐍𝐨𝐭𝐚!✍️ 🇻🇪 \nMin. de Ecosocialismo, @...",Tue Aug 03,01:46:08,1,Si,139,0,NaN,152870048,katyanicarolina,...,[],2,No,Twitter for Android,False,55246,[],[],0,[https://t.co/CZpTz7xDqU]
2,Los #TrasladosEduFiscal del regimen #SierraAma...,Tue Aug 03,01:45:53,1,Si,12,0,NaN,931643047930138624,MariaEditaCuev2,...,[],1,No,Twitter for Android,False,2007,"[TrasladosEduFiscal, SierraAmazonía]","[[22, 41], [54, 69]]",2,"[https://t.co/GTKVctFZOF, https://t.co/GnzbaWq..."
3,Recorrimos las obras del nuevo Contact Center ...,Tue Aug 03,01:45:48,1,Si,65,0,NaN,1365607806,AlmaFuerte40,...,[],1,No,Twitter for Android,False,336030,[],[],0,[https://t.co/8XwyfUyU0N]
4,Si sufriste o sufris violencia d genero La Lin...,Tue Aug 03,01:45:18,1,No,4,2,NaN,3153998255,silloia,...,[],0,silloia,Twitter for Android,False,9998,[],[],0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7499,Emocionante y patriotica conversacion telefoni...,Sat Jul 31,20:57:14,20,Si,365,0,NaN,1138517728338370560,PalomaMartnezS1,...,[],1,No,Twitter for Android,False,11940,[],[],0,[https://t.co/A4OPfpOBaE]
7500,#BazarCall es una amenaza detectada a principi...,Sat Jul 31,20:57:14,20,Si,7,0,NaN,710123736175783938,sectest9,...,[],1,No,auto is the only way it can be,False,2385974,"[BazarCall, malware]","[[15, 25], [119, 127]]",2,[https://t.co/b2wi67b2TU]
7501,#BazarCall es una amenaza detectada a principi...,Sat Jul 31,20:57:06,20,Si,7,0,NaN,1142424032794406912,CyberSecurityN8,...,[],1,No,forRetweeting,False,860417,"[BazarCall, malware]","[[15, 25], [119, 127]]",2,[https://t.co/b2wi67b2TU]
7502,#BazarCall es una amenaza detectada a principi...,Sat Jul 31,20:56:36,20,Si,7,0,NaN,571737915,JordiRobert1,...,[],1,No,Twitter for Android,False,4806,"[BazarCall, malware]","[[15, 25], [119, 127]]",2,[https://t.co/b2wi67b2TU]


Antes de guardar en csv, como el separador entre columnas que vamos a utilizar es '|', ya que trabajando con texto usar ',' o ';' como separador es inviable, nos aseguramos que no tenemos ningun '|' en el texto, si lo hay, lo sustituimos por un espacio

In [125]:
df['text'] = df['text'].apply(lambda x: re.sub('\|',' ', x))
df['text'] = df['text'].apply(lambda x: re.sub('\|',' ', x))

 He leído tweets con 'llamada telefonica' o 'linea telefónica' en tweets relacionados con la situación en Cuba. ¿cómo veis que los quitemos? o al menos que quede notificado...

 No se si se podrían eliminar aquellos tweets que "telefónica" sea un **adjetivo**.... sería mejor porque sino habra que  apuntar miles de regex


In [126]:
# Expresiones regulares
re_0 = "llamada\stelefónica" 
re_1 = "linea\stelefónica"
re_2 = "compañía\stelefónica" 
re_3 = "factura\stelefónica"
re_4 = "conversación\stelefónica"
re_5 = "la\sguía\stelefónica"
re_6 = "una\scita\s(via)|(de\smanera)\stelefonica"
re_7 = "por\svia\stelefonica"
re_8 = "(una)|(la)\sencuesta\stelefonica"


In [127]:
def not_enterprise(tweet, *args):
    """
    Funcion que dada un tweet busca si contiene alguna de las varias expresiones regualares dadas
    
    params: 
        tweet (string): texto que contiene el tweet
        args* (string): Expresiones regulares a considerar
        
    return: Boolean
        True si el tweet contiene al menos una de las expresiones regulares, False en caso contrario
    """
    return any(re.search(regex, tweet) for regex in args)

In [128]:
df['not_enterprise'] = df['text'].apply(lambda x: not_enterprise(x, re_0, re_1, re_2, re_3, re_4, re_5, re_6, re_7, re_8))

In [129]:
df

,text,created_at,created_at_time,created_at_hour,retweeted,retweet_count,favorite_count,user_ verified,user_id,user_name,...,mentions_in_tweet,is_reply,source_device,finished_tweet,status_count,hashtags_text,hastags_indices,hastags_in_tweet,URLs,not_enterprise
0,"🗣️ ¡𝐓𝐨𝐦𝐚 𝐍𝐨𝐭𝐚!✍️ 🇻🇪 \nMin. de Ecosocialismo, @...",Tue Aug 03,01:46:40,1,Si,139,0,NaN,1386447251380359176,NoheGuzman4,...,2,No,Twitter for Android,False,3327,[],[],0,[https://t.co/CZpTz7xDqU],False
1,"🗣️ ¡𝐓𝐨𝐦𝐚 𝐍𝐨𝐭𝐚!✍️ 🇻🇪 \nMin. de Ecosocialismo, @...",Tue Aug 03,01:46:08,1,Si,139,0,NaN,152870048,katyanicarolina,...,2,No,Twitter for Android,False,55246,[],[],0,[https://t.co/CZpTz7xDqU],False
2,Los #TrasladosEduFiscal del regimen #SierraAma...,Tue Aug 03,01:45:53,1,Si,12,0,NaN,931643047930138624,MariaEditaCuev2,...,1,No,Twitter for Android,False,2007,"[TrasladosEduFiscal, SierraAmazonía]","[[22, 41], [54, 69]]",2,"[https://t.co/GTKVctFZOF, https://t.co/GnzbaWq...",False
3,Recorrimos las obras del nuevo Contact Center ...,Tue Aug 03,01:45:48,1,Si,65,0,NaN,1365607806,AlmaFuerte40,...,1,No,Twitter for Android,False,336030,[],[],0,[https://t.co/8XwyfUyU0N],False
4,Si sufriste o sufris violencia d genero La Lin...,Tue Aug 03,01:45:18,1,No,4,2,NaN,3153998255,silloia,...,0,silloia,Twitter for Android,False,9998,[],[],0,[],False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7499,Emocionante y patriotica conversacion telefoni...,Sat Jul 31,20:57:14,20,Si,365,0,NaN,1138517728338370560,PalomaMartnezS1,...,1,No,Twitter for Android,False,11940,[],[],0,[https://t.co/A4OPfpOBaE],False
7500,#BazarCall es una amenaza detectada a principi...,Sat Jul 31,20:57:14,20,Si,7,0,NaN,710123736175783938,sectest9,...,1,No,auto is the only way it can be,False,2385974,"[BazarCall, malware]","[[15, 25], [119, 127]]",2,[https://t.co/b2wi67b2TU],True
7501,#BazarCall es una amenaza detectada a principi...,Sat Jul 31,20:57:06,20,Si,7,0,NaN,1142424032794406912,CyberSecurityN8,...,1,No,forRetweeting,False,860417,"[BazarCall, malware]","[[15, 25], [119, 127]]",2,[https://t.co/b2wi67b2TU],True
7502,#BazarCall es una amenaza detectada a principi...,Sat Jul 31,20:56:36,20,Si,7,0,NaN,571737915,JordiRobert1,...,1,No,Twitter for Android,False,4806,"[BazarCall, malware]","[[15, 25], [119, 127]]",2,[https://t.co/b2wi67b2TU],True


Extraction:

In [131]:
df_extract = df[['text', 'retweeted', 'not_enterprise']]
#df_extract.to_csv('extraccion_1.csv', encoding='utf-8-sig', sep='|', index=False)
df_extract.to_excel('extraccion_1.xlsx')
df.to_excel('extraccion_1_full.xlsx')